# Club elo scraper
Store club elo per every 6 months as list of pandas dataframes
and store it as a csv file

In [3]:
from datetime import datetime
import io
import requests
import pandas as pd
from pathlib import Path
import time
import os

In [5]:
# Generate dates with 6-month intervals starting from 2010-01-01 to today
date_range = pd.date_range(start='2020-01-01', end=pd.Timestamp.now(), freq='6ME')

# Convert the dates to the desired string format 'YYYY-MM-DD'
date_list = date_range.strftime('%Y-%m-%d').tolist()

In [6]:

dfs = []
merged_df = pd.DataFrame()

for date in date_list:
    response = requests.get(f"https://api.clubelo.com/{date}")

    club_elo_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    club_elo_df['Rank'] = club_elo_df.index + 1
    club_elo_df['Updated'] = date
    
    dfs.append(club_elo_df)
    time.sleep(2)
    # merged_df.append(club_elo_df)
    
res = pd.concat(dfs)



In [7]:

res.head()

,Rank,Club,Country,Level,Elo,From,To,Updated
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31


## Changing name of club elos to that of clubs df
So rn, names are mixed up (Man city instead of Manchester City) etc
We have to find a way to change this name into that of clubs df

---
Below is IDK whats gonig on


In [8]:
# Now we want to change club name according to clubs.csv and also change the index column name
# columns = [x.lower() for x in res.columns]
new_rows = ['rank', 'name', 'country', 'level', 'elo', 'from', 'to', 'updated']

res.columns = new_rows
# res = res[0:].reset_index(drop=True)  # Remove the first row and reset index
# 
res.head()


,rank,name,country,level,elo,from,to,updated
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31


In [9]:
# Change name to that in transfer mrkt dataset


# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 
CLUBS_FILE = 'clubs.csv'
# res.to_csv(DATA_DIR / 'club_elos.csv')


filepath=os.path.join(DATA_DIR, CLUBS_FILE)
clubs_df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")

In [10]:
clubs_df.head(5)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,31,25.6,16,51.6,7,Tüpraş Stadyumu,42445,€-18.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,25.7,19,73.1,18,Olimpico di Roma,73261,€-65.60m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,25,25.3,18,72.0,16,Tottenham Hotspur Stadium,62850,€-93.55m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...


In [12]:
# Matching name
clubs_df[clubs_df['name'].str.contains('Liverpool')]

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
44,31,fc-liverpool,Liverpool FC,GB1,NaN,27,25.2,21,77.8,22,Anfield,60725,+€5.00m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-liverpool/s...


In [43]:
#

,rank,name,country,level,elo,from,to,updated,club_id
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31,-1
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31,-1
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31,-1
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31,-1
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31,-1


['rank', 'club', 'country', 'level', 'elo', 'from', 'to', 'updated']

In [ ]:

# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 

res.to_csv(DATA_DIR / 'club_elos.csv')